In [1]:
import pandas as pd
import numpy as np

In [2]:
csv_path = 'problems/pod2/data/'

In [3]:
# Read csv files into Pandas DataFrames and replace NaN values
def open_csv(filename):
    dateparse = lambda x: pd.datetime.strptime(x, '%d-%b-%Y %H:%M:%S')
    content = pd.read_csv(csv_path + filename, 
        parse_dates=['time'],
        date_parser=dateparse)
    content = content.interpolate(method='linear', limit_direction='both')
    return content

cooler01 = open_csv('cooler01.csv')
cooler02 = open_csv('cooler02.csv')
cooler03 = open_csv('cooler03.csv')
cooler04 = open_csv('cooler04.csv')
rack01 = open_csv('rack01.csv')
rack02 = open_csv('rack02.csv')
rack03 = open_csv('rack03.csv')
rack04 = open_csv('rack04.csv')
rack05 = open_csv('rack05.csv')
rack06 = open_csv('rack06.csv')
rack07 = open_csv('rack07.csv')
rack08 = open_csv('rack08.csv')
rack09 = open_csv('rack09.csv')
rack10 = open_csv('rack10.csv')

In [4]:
# Density of air at 1 atm and 30°C
rho = 1.177
# Kinematic viscosity of air
nu = 1.568e-5
# Thermal conductivity
k = 2.624e-5
# Prandtl number of air
Pr = 0.707

In [5]:
# Convert CRAC discharge air flow from kg/s to m³/s
P02HDZ01_Q = cooler01['mass_flow'] / rho
P02HDZ02_Q = cooler02['mass_flow'] / rho
P02HDZ03_Q = cooler03['mass_flow'] / rho
P02HDZ04_Q = cooler04['mass_flow'] / rho

In [6]:
# Discharge air temperatures
P02HDZ01_T = cooler01['temperature_discharge_'] * 0.85
P02HDZ02_T = cooler02['temperature_discharge_'] * 0.85
P02HDZ03_T = cooler03['temperature_discharge_'] * 0.85
P02HDZ04_T = cooler04['temperature_discharge_'] * 0.85

In [7]:
# Rack power usage is in three-phase
def sum_power(rack):
    return rack['realpower_l1'] + rack['realpower_l2'] + rack['realpower_l3']

P02R01_P = sum_power(rack01)
P02R02_P = sum_power(rack02)
P02R03_P = sum_power(rack03)
P02R04_P = sum_power(rack04)
P02R05_P = sum_power(rack05)
P02R06_P = sum_power(rack06)
P02R07_P = sum_power(rack07)
P02R08_P = sum_power(rack08)
P02R09_P = sum_power(rack09)
P02R10_P = sum_power(rack10)

In [8]:
# Calculate average fan rpm for all servers in the rack
def avg_fan_rpm(rack):
    rpm_cols = [col for col in rack.columns if col.startswith('speed_rpm')]
    return rack[rpm_cols].mean(axis='columns')

# Fan data is missing for racks 01-03
P02R04_RPM = avg_fan_rpm(rack04)
P02R05_RPM = avg_fan_rpm(rack05)
P02R06_RPM = avg_fan_rpm(rack06)
P02R07_RPM = avg_fan_rpm(rack07)
P02R08_RPM = avg_fan_rpm(rack08)
P02R09_RPM = avg_fan_rpm(rack09)
P02R10_RPM = avg_fan_rpm(rack10)

In [9]:
# Dell R430
# From Delta Electronics GFB0412SHS-DF00 datasheet
# Max input power (W)
Pmax_R430 = 13.2
# Max speed (RPM)
Nmax_R430 = 14300
# Operational speed (RPM)
Nop_R430 = 7500
# Max air flow (CFM)
Qmax_R430 = 30.23
# Number of fans in each server
Nfans_R430 = 6
# Operational power from cube law of fans
Pop = Pmax_R430 / (Nmax_R430 / Nop_R430)**3 * Nfans_R430
# Assume fan volumetric flow is proportional to power 
# around operating point
Qop_R430 = Qmax_R430 / (Pmax_R430 / Pop)
# Calculate flow in m^3/s per RPM at
Q_per_RPM_per_R430 =  Qop_R430 * 0.3048**3 / 60 / Nop_R430
# Calculate average server air flow rate. For simplicity use
# the mean fan speed for each server, multiplied by number of servers
# in each rack.
P02R04_Q = P02R04_RPM * Q_per_RPM_per_R430 * 26
P02R05_Q = P02R05_RPM * Q_per_RPM_per_R430 * 30
P02R06_Q = P02R06_RPM * Q_per_RPM_per_R430 * 30
P02R07_Q = P02R07_RPM * Q_per_RPM_per_R430 * 30

# Dell R530
# From Delta Electronics PFR0612DHE-SP00 datasheet
Pmax_R530 = 19.20
Nmax_R530 = 14500
Nop_R530 = 7500
Qmax_R530 = 66
Nfans_R530 = 6
Pop = Pmax_R530 / (Nmax_R530 / Nop_R530)**3 * Nfans_R530
Qop_R530 = Qmax_R530 / (Pmax_R530/Pop)
# The simulation is closer to experiment when fan flow rate is
# increased by 10%.
Q_per_RPM_per_R530 =  Qop_R530 * 0.3048**3 / 60 / Nop_R530 * 1.10
P02R08_Q = P02R08_RPM * Q_per_RPM_per_R530 * 16
P02R09_Q = P02R09_RPM * Q_per_RPM_per_R530 * 16
P02R10_Q = P02R10_RPM * Q_per_RPM_per_R530 * 8

# 3 x HP C7000 per rack
# RPM is missing for rack 01-03
# Simulation shows flow rate should be a bit more than half 
# that of P02R05 per watt of power usage.
Q_per_P = P02R05_Q.mean(axis='rows') / P02R05_P.mean(axis='rows')
P02R01_Q = P02R01_P * Q_per_P * 0.50
P02R02_Q = P02R02_P * Q_per_P * 0.50
P02R03_Q = P02R03_P * Q_per_P * 0.50

In [10]:
# Calculate the expected temperature jump across the servers
def deltaT(p, q):
    return (p / 1000 * nu) / (q * k * Pr)

P02R01_T = deltaT(P02R01_P, P02R01_Q)
P02R02_T = deltaT(P02R02_P, P02R02_Q)
P02R03_T = deltaT(P02R03_P, P02R03_Q)
P02R04_T = deltaT(P02R04_P, P02R04_Q)
P02R05_T = deltaT(P02R05_P, P02R05_Q)
P02R06_T = deltaT(P02R06_P, P02R06_Q)
P02R07_T = deltaT(P02R07_P, P02R07_Q)
P02R08_T = deltaT(P02R08_P, P02R08_Q)
P02R09_T = deltaT(P02R09_P, P02R09_Q)
P02R10_T = deltaT(P02R10_P, P02R10_Q)

In [11]:
# Write to input csv file (use unix timestamp)
ts = cooler01['time'].astype(np.int64) // 10**9
output = pd.DataFrame({'time':ts,
    'P02HDZ01_T': P02HDZ01_T, 'P02HDZ01_Q': P02HDZ01_Q,
    'P02HDZ02_T': P02HDZ02_T, 'P02HDZ02_Q': P02HDZ02_Q,
    'P02HDZ03_T': P02HDZ03_T, 'P02HDZ03_Q': P02HDZ03_Q,
    'P02HDZ04_T': P02HDZ04_T, 'P02HDZ04_Q': P02HDZ04_Q,
    'P02R01_T': P02R01_T, 'P02R01_Q': P02R01_Q,
    'P02R02_T': P02R02_T, 'P02R02_Q': P02R02_Q,
    'P02R03_T': P02R03_T, 'P02R03_Q': P02R03_Q,
    'P02R04_T': P02R04_T, 'P02R04_Q': P02R04_Q,
    'P02R05_T': P02R05_T, 'P02R05_Q': P02R05_Q,
    'P02R06_T': P02R06_T, 'P02R06_Q': P02R06_Q,
    'P02R07_T': P02R07_T, 'P02R07_Q': P02R07_Q,
    'P02R08_T': P02R08_T, 'P02R08_Q': P02R08_Q,
    'P02R09_T': P02R09_T, 'P02R09_Q': P02R09_Q,
    'P02R10_T': P02R10_T, 'P02R10_Q': P02R10_Q})

output.to_csv(csv_path + 'input.csv', index=False, header=True)

In [12]:
# Print min/max values
all_Q = pd.concat([P02HDZ01_Q, P02HDZ02_Q, P02HDZ03_Q, P02HDZ04_Q, P02R01_Q, P02R02_Q, P02R03_Q, P02R04_Q, P02R05_Q, P02R06_Q, P02R07_Q, P02R08_Q, P02R09_Q, P02R10_Q])
all_T = pd.concat([P02HDZ01_T, P02HDZ02_T, P02HDZ03_T, P02HDZ04_T, P02R01_T, P02R02_T, P02R03_T, P02R04_T, P02R05_T, P02R06_T, P02R07_T, P02R08_T, P02R09_T, P02R10_T])
print(f'Q_max={max(all_Q)}, Q_min={min(all_Q)}, T_max={max(all_T)}, T_min={min(all_T)}')

Q_max=2.0486108354044434, Q_min=0.082842398208325, T_max=19.805, T_min=6.3963778230293435


In [14]:
output_max = pd.DataFrame({'time':ts,
    'P02HDZ01_T': max(P02HDZ01_T), 'P02HDZ01_Q': max(P02HDZ01_Q),
    'P02HDZ02_T': max(P02HDZ02_T), 'P02HDZ02_Q': max(P02HDZ02_Q),
    'P02HDZ03_T': max(P02HDZ03_T), 'P02HDZ03_Q': max(P02HDZ03_Q),
    'P02HDZ04_T': max(P02HDZ04_T), 'P02HDZ04_Q': max(P02HDZ04_Q),
    'P02R01_T': max(P02R01_T), 'P02R01_Q': max(P02R01_Q),
    'P02R02_T': max(P02R02_T), 'P02R02_Q': max(P02R02_Q),
    'P02R03_T': max(P02R03_T), 'P02R03_Q': max(P02R03_Q),
    'P02R04_T': max(P02R04_T), 'P02R04_Q': max(P02R04_Q),
    'P02R05_T': max(P02R05_T), 'P02R05_Q': max(P02R05_Q),
    'P02R06_T': max(P02R06_T), 'P02R06_Q': max(P02R06_Q),
    'P02R07_T': max(P02R07_T), 'P02R07_Q': max(P02R07_Q),
    'P02R08_T': max(P02R08_T), 'P02R08_Q': max(P02R08_Q),
    'P02R09_T': max(P02R09_T), 'P02R09_Q': max(P02R09_Q),
    'P02R10_T': max(P02R10_T), 'P02R10_Q': max(P02R10_Q)})

output_max.to_csv(csv_path + 'input_max.csv', index=False, header=True)